# Test loop closure

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go
from copy import deepcopy

import planeslam.io as io
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU, trajectory_plot_trace
from planeslam.geometry.util import quat_to_R

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

### Load AirSim data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_sub_2_1200_samples_5hz_noyaw', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', 'data', 'airsim', 'blocks_sub_2_1200_samples_5hz_noyaw', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Subsample data
sub_factor = 5
PC_data = PC_data[::sub_factor]
drone_positions = drone_positions[::sub_factor]
drone_orientations = drone_orientations[::sub_factor]

In [ ]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

drone_rotations = np.zeros((3,3,num_scans))
for i in range(num_scans):
    drone_rotations[:,:,i] = quat_to_R(drone_orientations[i])

In [ ]:
# Plot ground-truth trajectory
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(drone_positions)))
fig = go.Figure(data=gt_traj_trace)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Extract scans
num_scans = len(PC_data)
scans = num_scans * [None]
avg_extraction_time = 0
for i in range(num_scans):
    start_time = time.time()
    scans[i] = pc_to_scan(PC_data[i])
    scans[i].remove_small_planes(area_thresh=5.0)
    avg_extraction_time += time.time() - start_time
avg_extraction_time /= num_scans
print(avg_extraction_time)

### Open-loop registration (odometry)

In [ ]:
from planeslam.registration import robust_GN_register

R_abs = quat_to_R(drone_orientations[0])
t_abs = drone_positions[0,:].copy()
traj_est = np.zeros((num_scans, 3))
traj_est[0] = t_abs
traj_Rs = np.zeros((3, 3, num_scans))
traj_Rs[:,:,0] = R_abs
avg_runtime = 0

R_hats = []
t_hats = []

loop_closure_dist_thresh = 10.0
index_spacing_thresh = 10

r_errors = []

for i in range(1, num_scans):
    print("i = ", i)
    start_time = time.time()
    R_hat, t_hat = robust_GN_register(scans[i], scans[i-1])
    #R_hat = np.array([[0,1,0],[1,0,0],[0,0,1]]) @ R_hat.T @ np.array([[0,1,0],[1,0,0],[0,0,1]])
    t_abs += (R_abs @ t_hat).flatten()
    R_abs = R_hat @ R_abs
    avg_runtime += time.time() - start_time
    traj_est[i] = t_abs
    traj_Rs[:,:,i] = R_abs

    R_hats.append(R_hat)
    t_hats.append(t_hat)

    R_1 = quat_to_R(drone_orientations[i-1])
    R_2 = quat_to_R(drone_orientations[i])
    R_gt = R_2 @ R_1.T

    t_gt = drone_positions[i] - drone_positions[i-1]

    # print("  R_hat: \n", R_hat)
    # print("  R_gt: \n", R_gt)
    # print("  t_hat: ", t_hat.flatten())
    # print("  t_gt: ", t_gt)
    r_error = np.linalg.norm(np.eye(3) - R_hat @ R_gt.T)
    r_errors.append(r_error)
    #print("  relative rotational error: ", np.linalg.norm(np.eye(3) - R_hat @ R_gt.T))
    #print("  relative translational error: ", np.linalg.norm(t_hat - t_gt))

    # Loop closure detection
    # - check if t_abs is within loop_closure_dist_thresh of any previous traj_est[j] for i-j > some spacing threshold 
    # - run registration for scan[i], scan[j]: if rotational/translational error is low enough, then add (i,j) as loop closure
    loop_closure_dists = np.linalg.norm(t_abs - traj_est[:i], axis=1)
    #print(loop_closure_dists)
    loop_closure_mask = (loop_closure_dists < loop_closure_dist_thresh)
    #print(loop_closure_mask)


avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

In [ ]:
data = go.Scatter(x=np.arange(len(r_errors)), y=r_errors)
fig = go.Figure(data=data)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(drone_positions)), name="Ground-truth")
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(traj_est)), name="Estimated")
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'), legend=dict(yanchor="top", y=0.99, xanchor="right", x=0.99))
fig.show()

In [ ]:
gt_traj_trace_2d = go.Scatter(x=drone_positions[:,0], y=drone_positions[:,1], mode='lines+markers',
    marker=dict(size=5))
gt_traj_trace_2d.name = "Ground-truth"
est_traj_trace_2d = go.Scatter(x=traj_est[:,0], y=traj_est[:,1], mode='lines+markers',
    marker=dict(size=5))
est_traj_trace_2d.name = "Estimated"
fig = go.Figure(data=[gt_traj_trace_2d, est_traj_trace_2d])
fig.update_layout(width=1500, height=600, scene=dict(aspectmode='data'),
    legend=dict(yanchor="bottom", y=0.03, xanchor="right", x=0.99))
fig.show()

In [ ]:
est_data = trajectory_plot_trace(traj_Rs, traj_est)
gt_data = trajectory_plot_trace(drone_rotations, drone_positions, color='blue')
fig = go.Figure(data=est_data+gt_data)
fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
np.sum(np.linalg.norm(np.diff(drone_positions, axis=0), axis=1))

In [ ]:
# Translation RMSE
np.mean(np.linalg.norm(drone_positions - traj_est, axis=1))

In [ ]:
np.std(np.linalg.norm(drone_positions - traj_est, axis=1))

In [ ]:
from planeslam.geometry.util import R_to_quat

# Rotational error
avg_rot_errs = []

for i in range(num_scans):
    R1 = drone_rotations[:,:,i]
    R2 = traj_Rs[:,:,i]

    q_diff = R_to_quat(R2 @ R1.T)
    theta = np.degrees(np.arctan2(np.linalg.norm(q_diff[:3]), q_diff[3]))

    avg_rot_errs.append(theta)

print(np.mean(avg_rot_errs))
print(np.std(avg_rot_errs))

In [ ]:
theta

Form pose graph

In [ ]:
from graphslam.graph import Graph
from graphslam.vertex import Vertex
from graphslam.edge.edge_odometry import EdgeOdometry
from graphslam.pose.se3 import PoseSE3

from planeslam.geometry.util import R_to_quat

In [ ]:
vertices = []
edges = []

# Add first vertex
p = PoseSE3(traj_est[0], R_to_quat(traj_Rs[:,:,0]))
v = Vertex(0, p)
vertices.append(v)

# For each scan
for i in range(1, num_scans):
    # Add new vertex
    p = PoseSE3(traj_est[i], R_to_quat(traj_Rs[:,:,i]))
    v = Vertex(i, p)
    vertices.append(v)

    # Add odometry edge
    information = np.eye(6)
    estimate = PoseSE3(t_hats[i-1], R_to_quat(R_hats[i-1]))
    estimate.normalize()
    e = EdgeOdometry([i-1, i], information, estimate)
    edges.append(e)

In [ ]:
from planeslam.registration import loop_closure_register

# Add loop closure edges
loop_closures = [(0, 59)]

for (i,j) in loop_closures:
    R_hat, t_hat = loop_closure_register(scans[j], scans[i], (traj_Rs[:,:,j], traj_est[j]), (traj_Rs[:,:,i], traj_est[i]), t_loss_thresh=0.1)
    information = np.eye(6)
    estimate = PoseSE3(t_hat, R_to_quat(R_hat))
    estimate.normalize()
    e = EdgeOdometry([i, j], information, estimate)
    edges.append(e)

In [ ]:
g = Graph(edges, vertices)

In [ ]:
g.optimize()

In [ ]:
g._edges

In [ ]:
positions = np.zeros((num_scans, 3))
for i, v in enumerate(g._vertices):
    positions[i] = v.pose.position

In [ ]:
rotations = np.zeros((3, 3, num_scans))
for i, v in enumerate(g._vertices):
    rotations[:,:,i] = quat_to_R(v.pose.orientation)

In [ ]:
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(drone_positions)), name="Ground-truth")
est_traj_trace = go.Scatter3d(x=positions[:,0], y=positions[:,1], z=positions[:,2], 
    marker=dict(size=5), hovertext=np.arange(len(traj_est)), name="Estimated")
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'), legend=dict(yanchor="top", y=0.7, xanchor="left", x=0.1))
fig.show()

In [ ]:
# est_data = trajectory_plot_trace(rotations, positions)
# gt_data = trajectory_plot_trace(drone_rotations, drone_positions, color='blue')
# fig = go.Figure(data=est_data+gt_data)
# fig.update_layout(width=1600, height=900, scene=dict(aspectmode='data'))
# fig.show()

In [ ]:
# RMSE error
np.mean(np.linalg.norm(positions - drone_positions, axis=1))

Map

In [ ]:
# Initialize transformed scans
scans_transformed = []
for i in range(num_scans):
    scans_transformed.append(deepcopy(scans[i]))
    scans_transformed[i].transform(rotations[:,:,i], positions[i])

In [ ]:
P_transformed = num_scans * [None]
for i in range(num_scans):
    P_transformed[i] = (drone_rotations[:,:,i] @ PC_data[i].T).T + drone_positions[i]

In [ ]:
from planeslam.general import pc_plot_trace

i = 50

fig = go.Figure(data=[pc_plot_trace(P_transformed[i])]+scans_transformed[i].plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Plot scans
fig = go.Figure()

for i, scan in enumerate(scans_transformed):
    for t in scan.plot_trace():
        fig.add_trace(t)

fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
merged = scans_transformed[0]

for s in scans_transformed[1:]:
    merged = merged.merge(s, dist_thresh=7.5)
    merged.reduce_inside(p2p_dist_thresh=5)
    merged.remove_small_planes(area_thresh=1.0)
    merged.fuse_edges(vertex_merge_thresh=2.0)

In [ ]:
# Plot merge
fig = go.Figure(data=merged.plot_trace(colors=['blue']))
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()